In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from dags.utils import decode_zip, extract_headers, decode_body 
# load_dotenv()
# # SCOPES: Gmail read-only
# SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
# token_path = os.getenv("token_path")
# credentials_path = os.getenv("credentials_path")

In [6]:
path_1 = "data/starred_20-08-2025-09-49.json.gz"
path_2 = "data/unimportant_20-08-2025-09-49.json.gz"
decompressed_data_1 = decode_zip(path_1)
decompressed_data_2 = decode_zip(path_2)

df_imp =pd.DataFrame(decompressed_data_1)
df_unimp =pd.DataFrame(decompressed_data_2)

In [ ]:
#1.5 seconds faster for 250 rows of data
df_imp[["Subject","Sender"]] = df_imp["Payload"].apply(lambda x: pd.Series(extract_headers(x)))
df_imp["Body"] = df_imp["Payload"].apply(decode_body)
df_imp.drop(["Payload"], axis=1, inplace=True)
df_unimp["Important"] = 0


df_unimp["Subject"] = df_unimp["Payload"].apply(extract_headers)
df_unimp["Body"] = df_unimp["Payload"].apply(decode_body)
df_unimp.drop(["Payload"], axis=1, inplace=True)
df_unimp["Important"] = 0
df_unimp.head(5)

,Id,Important,Subject,Sender,Body
0,198c257736d2c031,0,hey looking out for reccomendation on below of...,glassdoor community,get the latest tech india trending posts tech ...
1,198c2596e59da1b6,0,agentic ai developer at aritha consulting serv...,glassdoor jobs,sb infowaves is hiring job alert machine learn...
2,198c2124c1f7f8e7,0,data scientist at bloktrek,jushey from instahyre,hey satwik methari we have some career opportu...
3,198bc4ad4b9fb329,0,new activity in general announcements,freelancercom,new activity in general announcements new post...
4,198c0e9a42037805,0,bring home the chef vibes,zepp student purchase program,cook faster eat better from breakfast to dinne...


In [12]:
from dags.utils import extract_headers, decode_body 
for payload in decompressed_data_1["Payload"][7:8]:
    subject, sender = extract_headers(payload)
    body_text = decode_body(payload)

    print(f"\n--- Email ---")
    print(f"From: {sender}")
    print(f"Subject: {subject}")
    print(f"Body:\n{body_text}")


--- Email ---
From: 
Subject: hpcl 2025 cbt admit card
Body:
dear candidate this has reference to your application for the position of electrical engineer corresponding to hpcl recruitment notification you can download your admit card for your computer based test scheduled on 16th august 2025 by clicking on the following link for login please use below credentials user 202607010999 password date of birth ddmmyyyy note after 1st login please change the passwords or else please direct download from below link ht path please note that no request for change in test centre will be entertained you may however contact 02261087523 for any other queries mon to sat 900 am to 600 pm regards hpcl help desk


In [9]:
import torch
from multiprocessing import cpu_count
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
print(cpu_count())
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

16
PyTorch version: 2.8.0+cu129
CUDA available: True


In [11]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
out = tokenizer(body_text,
                truncation=True,       
                max_length=512,        
                padding="max_length", 
                return_tensors="pt",
                return_length = True)

In [13]:
print(out["length"])

tensor([512])
